In [1]:
import os
os.chdir("../")

In [2]:
import glob

mscx_path_list = []
for item in glob.glob("./xmls/mscx/*"):
    mscx_path_list.append(os.path.abspath(item))
mscx_path_list.sort()

tstree_path_list = []
for item in glob.glob("./xmls/tstree/*"):
    tstree_path_list.append(os.path.abspath(item))
tstree_path_list.sort()

In [3]:
from alm.comparator import tree_similarity_calculator as TSCalculator
import pprint

tree_similarities = TSCalculator.calc_tree_similarities(mscx_path_list, tstree_path_list)

res = []
for item in tree_similarities:
    res.append([item.song_name, item.denominator, item.numerator, item.calc_similarity()])

pprint.pprint(res)

[['BE_FREE_A', 45, 10],
 ['BE_FREE_S', 53, 9],
 ['SUN_SHINE!!!_A', 99, 15],
 ['SUN_SHINE!!!_S', 58, 8],
 ['beautiful_days_A', 51, 11],
 ['beautiful_days_S', 60, 8],
 ['no_more_war_A', 121, 18],
 ['no_more_war_S', 33, 3],
 ['たけてん_A', 51, 10],
 ['たけてん_S', 77, 10],
 ['オレンジ_A', 75, 6],
 ['オレンジ_S', 114, 30],
 ['キセキ_A', 54, 8],
 ['キセキ_S', 81, 13],
 ['サヨナラから始めよう_A', 112, 13],
 ['サヨナラから始めよう_S', 36, 2],
 ['ビリーヴ_A', 74, 22],
 ['ビリーヴ_S', 82, 16],
 ['ルーキーズ_A', 105, 19],
 ['ルーキーズ_S', 101, 16],
 ['人_A', 111, 15],
 ['人_S', 77, 9],
 ['刹那_A', 47, 3],
 ['刹那_S', 48, 4],
 ['君想い_A', 109, 18],
 ['君想い_S', 71, 9],
 ['愛唄_A', 74, 12],
 ['愛唄_S', 57, 13],
 ['扉_A', 87, 18],
 ['扉_S', 54, 10],
 ['旅立ち_A', 57, 12],
 ['旅立ち_S', 58, 10],
 ['星影のエール_A', 58, 12],
 ['星影のエール_S', 65, 9],
 ['涙空_A', 113, 9],
 ['涙空_S', 29, 8],
 ['花唄_A', 87, 18],
 ['花唄_S', 55, 13],
 ['遥か_A', 95, 21],
 ['遥か_S', 63, 10]]
